In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#minha funcao para mostrar as imagens
def imshow(im,big=False):
    if big:
        plt.figure(figsize=(20,20))
    else:
        plt.figure()
    if len(im.shape)<3:
        plt.imshow(im, cmap='gray')#As cores ficam zoadas no Jupyter se deixar sem esse parâmetro
    else:
        plt.imshow(im[:,:,::-1]) 
    plt.axis('off')
    plt.show()

In [ ]:
def f2i(im):#float to int
    return np.round(im * 255.0).astype('uint8')
def i2f(im):#int to float
    return im.astype('float') / 255.0
def norm(im):
    return (im-im.min())/(im.max()-im.min())

### 1. Abrindo Imagem

In [ ]:
im = cv2.imread("2.bmp")
imshow(im)

### 2. Verdes

In [ ]:
im_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)

eh_verde = (im_hsv[:,:,0] > 40) & (im_hsv[:,:,0] < 75)
bem_saturado = (im_hsv[:,:,1] > 100)
suficientemente_claro = (im_hsv[:,:,2] > 65)

im_hsv[:,:,2] = (im_hsv[:,:,2] * eh_verde * bem_saturado * suficientemente_claro)
im_verde = cv2.cvtColor(im_hsv, cv2.COLOR_HSV2BGR)
imshow(im_verde, True)